In [92]:
#hide

## Import essential packages below

import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
from IPython.core.display import display,HTML
%matplotlib inline

In [93]:
dft_cases = pd.read_csv('data/SnapshotCases-28-July.csv')
dft_deaths = pd.read_csv('data/SnapshotDeaths-28-July.csv')

In [94]:
dft_cases.head()

,states,14-Mar-20,15-Mar-20,16-Mar-20,17-Mar-20,18-Mar-20,19-Mar-20,20-Mar-20,21-Mar-20,22-Mar-20,...,19-Jul-20,20-Jul-20,21-Jul-20,22-Jul-20,23-Jul-20,24-Jul-20,25-Jul-20,26-Jul-20,27-Jul-20,28-Jul-20
0,Andaman and Nicobar Islands,0,0,0,0,0,0,0,0,0,...,199,203,208,217,236,255,286,314,334,359
1,Andhra Pradesh,1,1,1,1,1,3,3,5,6,...,49650,53724,58668,64713,72711,80858,88671,96298,102349,110297
2,Arunachal Pradesh,0,0,0,0,0,0,0,0,0,...,740,790,858,949,991,1056,1126,1158,1239,1330
3,Assam,0,0,0,0,0,0,0,0,0,...,23898,24991,26671,27643,28690,29820,30985,32127,33475,34846
4,Bihar,0,0,0,0,0,0,0,0,2,...,26379,27455,28564,30066,31691,33511,36314,38919,41111,43591


In [95]:
dft_deaths.head()

,states,14-Mar-20,15-Mar-20,16-Mar-20,17-Mar-20,18-Mar-20,19-Mar-20,20-Mar-20,21-Mar-20,22-Mar-20,...,19-Jul-20,20-Jul-20,21-Jul-20,22-Jul-20,23-Jul-20,24-Jul-20,25-Jul-20,26-Jul-20,27-Jul-20,28-Jul-20
0,Andaman and Nicobar Islands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,Andhra Pradesh,0,0,0,0,0,0,0,0,0,...,642,696,758,823,884,933,985,1041,1090,1148
2,Arunachal Pradesh,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
3,Assam,0,0,0,0,0,0,0,0,0,...,61,62,68,70,74,80,81,83,90,92
4,Bihar,0,0,0,0,0,0,0,0,1,...,179,187,198,208,212,221,232,249,255,269


In [96]:
dt_today = '28-Jul-20'
dt_yday = '27-Jul-20'

In [97]:
#dft_cases.set_index('states')
#dft_deaths.set_index('states')
#dft_cases.head(), dft_deaths.head()

In [98]:
#hide

# Get the latest count of the total number of cases for each state across India, and also one from a day before

#Here past cases are a day earlier, but this can be changed
dfc_cases = dft_cases.groupby('states')[dt_today].sum()
dfc_deaths = dft_deaths.groupby('states')[dt_today].sum()
dfp_cases = dft_cases.groupby('states')[dt_yday].sum()
dfp_deaths = dft_deaths.groupby('states')[dt_yday].sum()

In [99]:
type(dfc_cases), dfc_cases

(pandas.core.series.Series,
 states
 Andaman and Nicobar Islands       359
 Andhra Pradesh                 110297
 Arunachal Pradesh                1330
 Assam                           34846
 Bihar                           43591
 Chandigarh                        934
 Daman and Diu                       0
 Delhi                          132275
 Goa                              5287
 Gujarat                         57982
 Haryana                         32876
 Himachal Pradesh                 2330
 Jammu and Kashmir               18879
 Jharkhand                        9563
 Karnataka                      107001
 Kerala                          20895
 Lakshadweep                         0
 Madhya Pradesh                  29217
 Maharashtra                    391440
 Manipur                          2317
 Meghalaya                         779
 Mizoram                           384
 Nagaland                         1460
 Orissa                          28107
 Pondicherry                

In [100]:
dfc_cases_series = pd.Series(dfc_cases, name="Cases")
dfc_deaths_series = pd.Series(dfc_deaths, name="Deaths")
dfp_cases_series = pd.Series(dfp_cases, name="PCases")
dfp_deaths_series = pd.Series(dfp_deaths, name="PDeaths")
df_table = pd.concat([dfc_cases_series, dfc_deaths_series, dfp_cases_series, dfp_deaths_series], axis=1)
df_table.head()

,Cases,Deaths,PCases,PDeaths
states,,,,
Andaman and Nicobar Islands,359,1,359,1
Andhra Pradesh,110297,1148,110297,1148
Arunachal Pradesh,1330,3,1330,3
Assam,34846,92,34846,92
Bihar,43591,269,43591,269


In [101]:
df_table.sort_values(by = ['Cases','Deaths'], ascending = [False, False], inplace=True)
df_table.head()

,Cases,Deaths,PCases,PDeaths
states,,,,
Maharashtra,391440,14164,391440,14164
Tamil Nadu,227688,3659,227688,3659
Delhi,132275,3881,132275,3881
Andhra Pradesh,110297,1148,110297,1148
Karnataka,107001,2064,107001,2064


In [102]:
df_table.reset_index(inplace=True)
df_table.head()

,states,Cases,Deaths,PCases,PDeaths
0,Maharashtra,391440,14164,391440,14164
1,Tamil Nadu,227688,3659,227688,3659
2,Delhi,132275,3881,132275,3881
3,Andhra Pradesh,110297,1148,110297,1148
4,Karnataka,107001,2064,107001,2064


In [103]:
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)
df_table['Fatality Rate'] = (100* df_table['Deaths']/ df_table['Cases']).round(2)

In [104]:
df_table.head()

,states,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate
0,Maharashtra,391440,14164,391440,14164,0,0,3.62
1,Tamil Nadu,227688,3659,227688,3659,0,0,1.61
2,Delhi,132275,3881,132275,3881,0,0,2.93
3,Andhra Pradesh,110297,1148,110297,1148,0,0,1.04
4,Karnataka,107001,2064,107001,2064,0,0,1.93
